## Speed up computation of Batchelor spectrum with Fortran

In [16]:
%load_ext fortranmagic

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic


In [5]:
import numpy as np
from epsilon_tools import *

In [6]:
def np_batchelor(k_rpm, chi, kb_rpm, p):
    '''
    Batchelor temperature gradient spectrum

    reference: Oakey, 1982
    see also: Lien, 1992
    '''
    import numpy as np
    import math
    from scipy.special import erfc

    a = np.sqrt(2 * p.q) * k_rpm / kb_rpm
    uppera = []
    for ai in a:
        uppera.append(erfc(ai / math.sqrt(2)) * math.sqrt(0.5 * math.pi))
    g = 2 * math.pi * a * (np.exp(-0.5 * a**2) - a * np.array(uppera))
    return math.sqrt(0.5 * p.q) * (chi / (kb_rpm* p.D)) * g / (2 * math.pi)

In [9]:
p = Parameters()
kbs = np.linspace(200,600)
f_cps = np.linspace(0,60,5000)
w=0.1
chi =1e-8
k_rpm = f_cps* 2 * np.pi/w

In [17]:
%%timeit
np_batchelor(k_rpm, chi, 350, p)

8.53 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%timeit
np_kraichnan(k_rpm, chi, 350, p)

59.4 µs ± 956 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
%%fortran
subroutine for_batchelor(k, chi, kb, q, D, s)
real, intent(in) :: k(:), chi, kb, q, D
real, dimension(size(k)) ,intent(out) :: s
real, dimension(size(k)) :: a, b, g
real :: pi
        
pi = 4*atan(1.)        
a = sqrt(2.*q) * k / kb

do i = 1, size(k), 1
    b(i) = erfc(a(i) / sqrt(2.)) * sqrt(0.5 * pi)
end do
g = 2 * pi * a * (exp(-0.5 * a**2) - a * b)
s = sqrt(0.5 * q) * (chi / (kb* D)) * g / (2*pi)
end subroutine

In [ ]:
def np_kraichnan(k_rpm, chi, kb_rpm, p):
    '''
    Kraichnan temperature gradient spectrum

    adapted from: Goto et al., 2016
    '''
    import numpy as np
    import math
    yk = math.sqrt(p.qk)* k_rpm / kb_rpm
    nom = chi*math.sqrt(p.qk)*yk*np.exp(-math.sqrt(6)*yk)
    denom = (p.D*kb_rpm)
    return nom/denom

In [58]:
%%fortran
subroutine for_kraichnan(k, chi, kb, qk, D, s)
real, intent(in) :: k(:), chi, kb, qk, D
real, dimension(size(k)) ,intent(out) :: s
real, dimension(size(k)) :: yk
real :: pi
        
pi = 4*atan(1.)        
yk(:) = sqrt(2.*qk) * k(:) / kb
s(:) = chi*sqrt(qk)*yk(:)*exp(-sqrt(6.)*yk(:))/(D*kb)

end subroutine

In [59]:
%%timeit
for_kraichnan(k_rpm, chi, 350, p.qk, p.D)

105 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
%%timeit
for_batchelor(k_rpm, chi, 350, p.q, p.D)

299 µs ± 8.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
